# Flame Regime GAN Model
## Generating Oscillatory Flame Behavior with Deep Learning

A Generative Adversarial Network that creates realistic oscillatory flame sequences based on combustion parameters (φ - equivalence ratio, u - velocity).

### Key Features:
- **Oscillatory Patterns**: Realistic flame oscillations with physics-based dynamics
- **Conditional Generation**: Parameter-controlled output (φ, u)
- **LSTM Architecture**: Temporal sequence modeling
- **Cloud Ready**: Optimized for Google Colab GPU/CPU

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install torch matplotlib seaborn pandas numpy scikit-learn tqdm scipy -q

# Mount Google Drive (optional - for loading your own data)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

print("Setup completed!")

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import re, os, pickle
from tqdm import tqdm
from typing import Tuple, List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Model Definition

In [ ]:
# Copy the FlameGAN model code here
# (The complete code from flame_gan_colab.py would go here)

# For demonstration, I'll include a simplified version:
exec(open('/content/flame_gan_colab.py').read()) if os.path.exists('/content/flame_gan_colab.py') else print("Please upload flame_gan_colab.py to Colab")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt

class FlameDataset(Dataset):
    def __init__(self, sequences, conditions):
        self.sequences = torch.FloatTensor(sequences)
        self.conditions = torch.FloatTensor(conditions)
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.conditions[idx]

class Generator(nn.Module):
    def __init__(self, noise_dim=128, condition_dim=2, sequence_length=51, output_dim=3):
        super().__init__()
        self.sequence_length = sequence_length
        
        self.fc = nn.Sequential(
            nn.Linear(noise_dim + condition_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 768),
            nn.BatchNorm1d(768),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        self.lstm = nn.LSTM(768, 384, num_layers=3, batch_first=True, dropout=0.2)
        self.output = nn.Sequential(
            nn.Linear(384, 192),
            nn.ReLU(),
            nn.Linear(192, output_dim),
            nn.Tanh()
        )
        
    def forward(self, noise, conditions):
        x = torch.cat([noise, conditions], dim=1)
        x = self.fc(x)
        x = x.unsqueeze(1).repeat(1, self.sequence_length, 1)
        x, _ = self.lstm(x)
        return self.output(x)

class Discriminator(nn.Module):
    def __init__(self, sequence_length=51, input_dim=3, condition_dim=2):
        super().__init__()
        self.sequence_length = sequence_length
        
        self.lstm = nn.LSTM(input_dim + condition_dim, 256, num_layers=2, batch_first=True, dropout=0.3)
        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.4),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
    def forward(self, sequences, conditions):
        conditions_expanded = conditions.unsqueeze(1).repeat(1, self.sequence_length, 1)
        x = torch.cat([sequences, conditions_expanded], dim=2)
        _, (hidden, _) = self.lstm(x)
        return self.classifier(hidden[-1])

class FlameGAN:
    def __init__(self, device=device):
        self.device = device
        self.noise_dim = 128
        self.condition_dim = 2
        self.sequence_length = 51
        self.output_dim = 3
        
        self.generator = Generator(self.noise_dim, self.condition_dim, self.sequence_length, self.output_dim).to(device)
        self.discriminator = Discriminator(self.sequence_length, self.output_dim, self.condition_dim).to(device)
        
        self.g_optimizer = optim.Adam(self.generator.parameters(), lr=0.0001, betas=(0.5, 0.999), weight_decay=1e-5)
        self.d_optimizer = optim.Adam(self.discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999), weight_decay=1e-5)
        
        self.criterion = nn.BCELoss()
        self.sequence_scaler = MinMaxScaler(feature_range=(-0.9, 0.9))
        self.condition_scaler = StandardScaler()
        self.training_history = {'g_losses': [], 'd_losses': [], 'epochs': []}
        
    def create_oscillatory_data(self, n_samples=300):
        sequences = []
        conditions = []
        
        for _ in range(n_samples):
            phi = np.random.uniform(0.8, 1.2)
            u = np.random.uniform(0.2, 0.7)
            
            t = np.linspace(0, 10, self.sequence_length)
            
            # Enhanced oscillatory patterns based on regime
            if phi < 0.9 or (phi >= 1.0 and u < 0.3):  # Stable
                freq_base = 0.8 + 0.4 * phi
                amp_base = 0.3 + 0.2 * u
                noise_level = 0.05
            else:  # Unstable
                freq_base = 1.2 + 0.8 * phi
                amp_base = 0.5 + 0.4 * u
                noise_level = 0.1
            
            # Generate correlated oscillatory components
            x1 = amp_base * np.sin(freq_base * t + np.random.uniform(0, 2*np.pi))
            x1 += 0.2 * amp_base * np.sin(3 * freq_base * t) + noise_level * np.random.randn(len(t))
            
            x2 = 0.8 * amp_base * np.sin(freq_base * t + np.pi/3)
            x2 += 0.15 * amp_base * np.sin(2 * freq_base * t) + noise_level * np.random.randn(len(t))
            
            x3 = 0.6 * amp_base * np.sin(freq_base * t - np.pi/4)
            x3 += 0.1 * amp_base * np.sin(1.5 * freq_base * t) + noise_level * np.random.randn(len(t))
            
            # Add regime-specific modulation
            if phi >= 1.1 and u >= 0.5:  # Highly unstable
                modulation = 0.3 * np.sin(0.2 * freq_base * t)
                x1 *= (1 + modulation)
                x2 *= (1 + 0.8 * modulation)
                x3 *= (1 + 0.6 * modulation)
            
            sequence = np.column_stack([x1, x2, x3])
            sequences.append(sequence)
            conditions.append([phi, u])
        
        sequences = np.array(sequences)
        conditions = np.array(conditions)
        
        # Normalize data
        seq_shape = sequences.shape
        sequences_reshaped = sequences.reshape(-1, seq_shape[-1])
        sequences_normalized = self.sequence_scaler.fit_transform(sequences_reshaped)
        sequences = sequences_normalized.reshape(seq_shape)
        
        conditions = self.condition_scaler.fit_transform(conditions)
        
        return sequences, conditions
    
    def train(self, sequences, conditions, epochs=200, batch_size=32, save_interval=50):
        dataset = FlameDataset(sequences, conditions)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        print(f"Training for {epochs} epochs on {len(sequences)} samples")
        
        for epoch in range(epochs):
            g_losses, d_losses = [], []
            
            for real_sequences, real_conditions in dataloader:
                batch_size = real_sequences.size(0)
                real_sequences = real_sequences.to(self.device)
                real_conditions = real_conditions.to(self.device)
                
                # Train Discriminator
                self.d_optimizer.zero_grad()
                
                # Real data
                real_labels = torch.ones(batch_size, 1).to(self.device) * 0.9  # Label smoothing
                real_output = self.discriminator(real_sequences, real_conditions)
                d_loss_real = self.criterion(real_output, real_labels)
                
                # Fake data
                noise = torch.randn(batch_size, self.noise_dim).to(self.device)
                fake_sequences = self.generator(noise, real_conditions)
                fake_labels = torch.zeros(batch_size, 1).to(self.device)
                fake_output = self.discriminator(fake_sequences.detach(), real_conditions)
                d_loss_fake = self.criterion(fake_output, fake_labels)
                
                d_loss = (d_loss_real + d_loss_fake) / 2
                d_loss.backward()
                self.d_optimizer.step()
                
                # Train Generator
                self.g_optimizer.zero_grad()
                fake_output = self.discriminator(fake_sequences, real_conditions)
                g_loss = self.criterion(fake_output, torch.ones(batch_size, 1).to(self.device))
                g_loss.backward()
                self.g_optimizer.step()
                
                g_losses.append(g_loss.item())
                d_losses.append(d_loss.item())
            
            avg_g_loss = np.mean(g_losses)
            avg_d_loss = np.mean(d_losses)
            
            self.training_history['g_losses'].append(avg_g_loss)
            self.training_history['d_losses'].append(avg_d_loss)
            self.training_history['epochs'].append(epoch)
            
            if epoch % 25 == 0:
                print(f"Epoch [{epoch}/{epochs}] - G Loss: {avg_g_loss:.4f}, D Loss: {avg_d_loss:.4f}")
            
            if epoch % save_interval == 0 and epoch > 0:
                self.save_model(f'/content/flame_gan_epoch_{epoch}.pth')
        
        print("Training completed!")
    
    def generate_sequence(self, phi, u):
        self.generator.eval()
        with torch.no_grad():
            condition = self.condition_scaler.transform([[phi, u]])
            condition_tensor = torch.FloatTensor(condition).to(self.device)
            noise = torch.randn(1, self.noise_dim).to(self.device)
            
            fake_sequence = self.generator(noise, condition_tensor)
            sequence = fake_sequence.cpu().numpy()[0]
            
            # Denormalize
            sequence = self.sequence_scaler.inverse_transform(sequence)
            
        self.generator.train()
        return sequence
    
    def save_model(self, path):
        torch.save({
            'generator': self.generator.state_dict(),
            'discriminator': self.discriminator.state_dict(),
            'g_optimizer': self.g_optimizer.state_dict(),
            'd_optimizer': self.d_optimizer.state_dict(),
            'sequence_scaler': self.sequence_scaler,
            'condition_scaler': self.condition_scaler,
            'training_history': self.training_history
        }, path)
    
    def load_model(self, path):
        checkpoint = torch.load(path, map_location=self.device)
        self.generator.load_state_dict(checkpoint['generator'])
        self.discriminator.load_state_dict(checkpoint['discriminator'])
        self.g_optimizer.load_state_dict(checkpoint['g_optimizer'])
        self.d_optimizer.load_state_dict(checkpoint['d_optimizer'])
        self.sequence_scaler = checkpoint['sequence_scaler']
        self.condition_scaler = checkpoint['condition_scaler']
        self.training_history = checkpoint['training_history']
    
    def plot_training_history(self):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(self.training_history['epochs'], self.training_history['g_losses'], label='Generator')
        plt.plot(self.training_history['epochs'], self.training_history['d_losses'], label='Discriminator')
        plt.title('Training Losses')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.plot(self.training_history['epochs'][-50:], self.training_history['g_losses'][-50:], label='Generator (Last 50)')
        plt.plot(self.training_history['epochs'][-50:], self.training_history['d_losses'][-50:], label='Discriminator (Last 50)')
        plt.title('Recent Training Progress')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

print("Optimized FlameGAN model loaded!")

## 3. Quick Demo with Synthetic Data

In [ ]:
# Quick demonstration with synthetic oscillatory data
print("Creating optimized Flame GAN...")
gan = FlameGAN(device=device)

print("Generating enhanced synthetic data...")
sequences, conditions = gan.create_oscillatory_data(n_samples=300)
print(f"Generated {len(sequences)} sequences with shape {sequences.shape}")

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
for i in range(4):
    ax = axes[i//2, i%2]
    seq = sequences[i]
    cond = conditions[i]
    
    # Denormalize for display
    seq_display = gan.sequence_scaler.inverse_transform(seq)
    cond_display = gan.condition_scaler.inverse_transform(cond.reshape(1, -1))[0]
    
    ax.plot(seq_display[:, 0], label='X1', linewidth=2, alpha=0.8)
    ax.plot(seq_display[:, 1], label='X2', linewidth=2, alpha=0.8)
    ax.plot(seq_display[:, 2], label='X3', linewidth=2, alpha=0.8)
    
    regime = "Stable" if cond_display[0] < 0.9 or (cond_display[0] >= 1.0 and cond_display[1] < 0.3) else "Unstable"
    ax.set_title(f'{regime}: φ={cond_display[0]:.2f}, u={cond_display[1]:.2f}', fontweight='bold')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Amplitude')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Enhanced Synthetic Oscillatory Data', y=1.02, fontsize=14, fontweight='bold')
plt.show()

## 4. Training the GAN

In [ ]:
print("Training optimized GAN...")
gan.train(sequences, conditions, epochs=200, batch_size=32, save_interval=50)
print("Training completed!")

In [ ]:
# Plot training history
gan.plot_training_history()

## 5. Generate New Sequences

In [ ]:
# Generate sequences for different flame conditions
test_conditions = [
    (0.8, 0.2, "Stable - Low φ, Low u"),
    (0.9, 0.35, "Stable - Medium φ, Medium u"),  
    (1.0, 0.5, "Transition"),
    (1.2, 0.6, "Unstable - High φ, High u"),
]

print("Generating sequences for different flame conditions...")

fig, axes = plt.subplots(len(test_conditions), 3, figsize=(16, 3*len(test_conditions)))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
components = ['X1', 'X2', 'X3']

for i, (phi, u, description) in enumerate(test_conditions):
    sequence = gan.generate_sequence(phi, u)
    print(f"Generated: φ={phi}, u={u}")
    
    for j in range(3):
        ax = axes[i, j] if len(test_conditions) > 1 else axes[j]
        ax.plot(sequence[:, j], color=colors[j], linewidth=2.5, alpha=0.8)
        ax.set_title(f'{description} - {components[j]}', fontweight='bold')
        ax.set_xlabel('Time Step')
        ax.set_ylabel('Amplitude')
        ax.grid(True, alpha=0.3)
        
        # Statistics
        mean_val, std_val = np.mean(sequence[:, j]), np.std(sequence[:, j])
        ax.text(0.02, 0.98, f'μ={mean_val:.3f}\nσ={std_val:.3f}', 
               transform=ax.transAxes, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.suptitle('Generated Oscillatory Flame Sequences', y=1.01, fontsize=16, fontweight='bold')
plt.show()

## 6. Analysis and Comparison

In [ ]:
# Analyze frequency content of generated sequences
from scipy.fft import fft, fftfreq

def analyze_frequency(sequence, sampling_rate=1.0):
    """Analyze frequency content of a sequence"""
    n = len(sequence)
    fft_vals = fft(sequence)
    freqs = fftfreq(n, 1/sampling_rate)
    pos_mask = freqs > 0
    return freqs[pos_mask], np.abs(fft_vals[pos_mask])

# Analyze frequency content for different regimes
fig, axes = plt.subplots(2, 3, figsize=(16, 8))

stable_seq = gan.generate_sequence(0.8, 0.3)
unstable_seq = gan.generate_sequence(1.2, 0.6)

sequences = [stable_seq, unstable_seq]
labels = ['Stable (φ=0.8, u=0.3)', 'Unstable (φ=1.2, u=0.6)']
components = ['X1', 'X2', 'X3']

for i, (seq, label) in enumerate(zip(sequences, labels)):
    for j, comp in enumerate(components):
        freqs, fft_vals = analyze_frequency(seq[:, j])
        axes[i, j].semilogy(freqs[:20], fft_vals[:20], linewidth=2.5, alpha=0.8)
        axes[i, j].set_title(f'{label} - {comp}')
        axes[i, j].set_xlabel('Frequency')
        axes[i, j].set_ylabel('Magnitude (log)')
        axes[i, j].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Frequency Analysis', y=1.02, fontsize=14)
plt.show()

In [ ]:
# Phase space analysis
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Generate sequences for phase space analysis
stable_seq = gan.generate_sequence(0.8, 0.2)
unstable_seq = gan.generate_sequence(1.2, 0.7)

phase_pairs = [(0, 1, 'X1', 'X2'), (0, 2, 'X1', 'X3'), (1, 2, 'X2', 'X3')]

for i, (x_idx, y_idx, x_label, y_label) in enumerate(phase_pairs):
    axes[i].scatter(stable_seq[:, x_idx], stable_seq[:, y_idx], 
                   alpha=0.6, s=25, label='Stable', color='blue')
    axes[i].scatter(unstable_seq[:, x_idx], unstable_seq[:, y_idx], 
                   alpha=0.6, s=25, label='Unstable', color='red')
    axes[i].set_xlabel(x_label)
    axes[i].set_ylabel(y_label)
    axes[i].set_title(f'{x_label} vs {y_label} Phase Space')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Phase Space Analysis', y=1.02, fontsize=14)
plt.show()

## 7. Parameter Study

In [ ]:
# Parameter sweep to study the effect of phi and u
phi_values = np.linspace(0.8, 1.2, 5)
u_values = np.linspace(0.2, 0.7, 4)

# Create a grid to visualize parameter effects
fig, axes = plt.subplots(len(u_values), len(phi_values), figsize=(18, 12))

print("Parameter study in progress...")

for i, u in enumerate(u_values):
    for j, phi in enumerate(phi_values):
        # Generate sequence
        sequence = gan.generate_sequence(phi, u)
        
        ax = axes[i, j]
        ax.plot(sequence[:, 0], linewidth=2, color='#FF6B6B', alpha=0.8)
        
        # Determine regime
        regime = "S" if phi < 0.9 or (phi >= 1.0 and u < 0.3) else "U"
        ax.set_title(f'φ={phi:.1f}, u={u:.1f} ({regime})', fontsize=9)
        ax.set_xlabel('Time')
        ax.set_ylabel('X1')
        ax.grid(True, alpha=0.3)
        
        # Background color by regime
        bg_color = '#e8f4fd' if regime == "S" else '#fde8e8'
        ax.set_facecolor(bg_color)

plt.tight_layout()
plt.suptitle('Parameter Study: φ and u Effects on Flame Oscillations', 
             y=0.98, fontsize=14, fontweight='bold')
plt.show()

print("Parameter study completed!")

## 8. Save Model and Results

In [ ]:
model_path = '/content/flame_gan_model.pth'
gan.save_model(model_path)
print(f"Model saved: {model_path}")

try:
    drive_path = '/content/drive/MyDrive/flame_gan_model.pth'
    gan.save_model(drive_path)
    print(f"Model also saved to Drive: {drive_path}")
except Exception as e:
    print(f"Drive save failed: {str(e)}")

In [ ]:
# Generate a dataset of sequences for different conditions
print("Generating export dataset...")

# Define parameter grid
phi_grid = [0.8, 0.9, 1.0, 1.1, 1.2]
u_grid = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
generated_data = []

for phi in phi_grid:
    for u in u_grid:
        # Generate sequence
        sequence = gan.generate_sequence(phi, u)
        filename = f"Phi_{phi:.1f}_u_{u:.1f}_generated.txt".replace('.', 'p')
        
        # Save sequence file
        np.savetxt(f'/content/{filename}', sequence, fmt='%.6E', delimiter='\t')
        
        # Collect metadata
        regime = 'stable' if phi < 0.9 or (phi >= 1.0 and u < 0.3) else 'unstable'
        generated_data.append({
            'filename': filename,
            'phi': phi,
            'u': u,
            'regime': regime,
            'x1_mean': np.mean(sequence[:, 0]),
            'x1_std': np.std(sequence[:, 0]),
            'x2_mean': np.mean(sequence[:, 1]),
            'x2_std': np.std(sequence[:, 1]),
            'x3_mean': np.mean(sequence[:, 2]),
            'x3_std': np.std(sequence[:, 2])
        })

# Create summary dataframe
summary_df = pd.DataFrame(generated_data)

# Save summary
summary_df.to_csv('/content/generated_sequences_summary.csv', index=False)

print(f"Generated {len(generated_data)} sequences")
print("Summary saved to generated_sequences_summary.csv")

# Display summary
print("\nDataset Summary:")
print(summary_df.groupby('regime').agg({
    'x1_std': 'mean',
    'x2_std': 'mean', 
    'x3_std': 'mean'
}).round(4))

## 9. Usage Guide

### Quick Generation:
```python
# Load trained model
gan = FlameGAN()
gan.load_model('/content/flame_gan_model.pth')

# Generate sequences
sequence = gan.generate_sequence(phi=1.0, u=0.5)
```

### Batch Generation:
```python
conditions = [(0.8, 0.3), (1.0, 0.5), (1.2, 0.7)]
for phi, u in conditions:
    seq = gan.generate_sequence(phi, u)
    print(f"φ={phi}, u={u}: regime={'stable' if phi<0.9 else 'unstable'}")
```

### Using Your Data:
1. Upload data to Google Drive
2. Mount Drive in cell 3
3. Replace synthetic data with your files
4. Retrain: `gan.train(your_sequences, your_conditions, epochs=250)`

### Files Generated:
- `flame_gan_model.pth` - Trained model
- `Phi_*_u_*_generated.txt` - Sequence files  
- `generated_sequences_summary.csv` - Summary statistics